Import Stuff

In [13]:
import ssl
import re


from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from umap import UMAP

from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from datetime import datetime
from textblob import TextBlob
from textblob_de import TextBlobDE
from datasets import load_dataset

import pickle
import importlib
import functions
import requests





In [54]:
from functions import *
importlib.reload(functions)

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

parties = ["CDU", "SPD", "FDP", "AFD", "LEFT", "GREENS"]
file_path = "C:/Users/Jacob/OneDrive/uni/MA WiSoz/Semester III/Computational Social Sciences/foPra/data/"


# Open the file in read mode
with open("API Keys/DeepL.txt", 'r', encoding='utf-8') as file:
    # Read the contents of the file
    api_key_deepl = file.read()

url = 'https://api-free.deepl.com/v2/translate'


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Edit Text documents

In [64]:
df_manifesto = pd.read_csv("data/parties/All_Manifestos.csv")
df_manifesto_tokenized = pd.read_csv("data/parties/All_Manifestos_tokenized.csv")
cmp_categories = pd.read_csv(file_path  + "cmp_categories.csv")
# df_manifesto["year"] = pd.to_numeric(df_manifesto["year"], errors='coerce')

In [60]:
def split_text_by_date(df, column="text", split_date=1998):
    new_rows = []
    for index, row in df.iterrows():
        if row['year'] < split_date:
            sentences = nltk.sent_tokenize(row[column])
            for i, sentence in enumerate(sentences):
                # Check if the sentence ends with a digit followed by a period
                if re.search(r'\d+\.$', sentence) and i + 1 < len(sentences) and sentences[i + 1][0].isupper():
                    sentence += ' ' + sentences[i + 1]
                    sentences[i + 1] = sentence
                    continue
                # Disregard abbreviations like "bzw.", "z.B.", "u.a."
                if re.search(r'\b(bzw|z\.B|u\.a)\.$', sentence):
                    new_row = row.copy()
                    new_row[column] = sentence
                    new_rows.append(new_row)
                    continue
                new_row = row.copy()
                new_row[column] = sentence
                new_rows.append(new_row)
        else:
            new_rows.append(row)
    # Filter out documents that do not have a space character and documents that are shorter than 6 characters
    new_rows = [row for row in new_rows if ' ' in row['text'] and len(row['text']) >= 6]
    return pd.DataFrame(new_rows)

def process_manifesto_data(df):
    """
    Processes the manifesto dataframe by performing various operations such as counting occurrences,
    mapping values, filtering rows, and merging text rows.

    Parameters:
    df (pd.DataFrame): The dataframe to process.

    Returns:
    pd.DataFrame: The processed dataframe.
    """
    df = split_text_by_date(df)
    # Count occurrences of distinct values in the party column
    party_counts = df["party"].value_counts()

    # Add a line that contains this count per party
    df["doc size of party"] = df["party"].map(party_counts)
    df["cmp_code"] = pd.to_numeric(df["cmp_code"], errors='coerce')
    df.index = range(1, len(df) + 1)
    
    # Remove all punctuation from the 'text' column
    df["text"] = df["text"].str.replace(r'[^\w\s]', '', regex=True)  # Delete entries that do not contain words in the text column
    df = df[df['text'].str.contains(r'\b\w+\b', na=False)]

    # Ensure the 'text' column contains only strings and handle NaN values
    df["text"] = df["text"].astype(str).fillna("")

    # Merge text rows
    df = merge_text_rows(df)

    df = split_text_by_date(df)

    return df

In [65]:
# Apply the function to df_manifesto
df_manifesto = process_manifesto_data(df_manifesto)
# df_manifesto_tokenized = process_manifesto_data(df_manifesto_tokenized)

In [51]:
df_manifesto = df_manifesto_final

In [46]:
def drop_entries_by_conditions(df):
    """
    Drops rows from the dataframe where year > 1997 and cmp_code is NaN.

    Parameters:
        df (pd.DataFrame): The input dataframe.

    Returns:
        pd.DataFrame: The filtered dataframe.
    """
    return df[ ~ ((df['cmp_code'].isna()) & (df['year'] > 1997))]

In [66]:
# Merge df_manifesto with cmp_categories to add the description_md column
df_manifesto = df_manifesto.merge(cmp_categories[['code', 'label', 'description_md']], 
                                  left_on='cmp_code', 
                                  right_on='code', 
                                  how='left')

# Drop the redundant 'code' column after the merge
df_manifesto.drop(columns=['code'], inplace=True)

In [50]:
# Apply the function to df_manifesto
df_manifesto = drop_na_after_date(df_manifesto, 1997)

In [ ]:
df_manifesto_final["sentiment"] = df_manifesto_final["text"].apply(lambda x: TextBlobDE(x).sentiment.polarity)

In [ ]:
test = TextBlobDE(df_manifesto_final["text"][2000])
test.sentiment.polarity


In [67]:
df_manifesto.to_pickle('data\df_manifesto.pkl')